In [1]:
from torf import Torrent
import hashlib
from math import ceil

In [12]:
tor.infohash

'54726d76b8fb9e7ed6be7ee1aa1fb4ce0b1b42bf'

In [13]:
print(tor.infohash, tor2.infohash)
print(tor.pieces, tor2.pieces)
print(tor.size, tor2.size)
print(tor.files, tor2.files)

54726d76b8fb9e7ed6be7ee1aa1fb4ce0b1b42bf 06de87bdfd7b978ac410a210d121018095cb5247
5 6
2185794 3083740
[File('pic.jpg', size=2185794)] [File('wallpapers\\turquoise_mountains_art_4k_hd_turquoise.jpg', size=2185794), File('wallpapers\\wallpaper.jpg', size=897946)]


In [14]:
for file in tor2.files:
    print(file.name, file.size)

turquoise_mountains_art_4k_hd_turquoise.jpg 2185794
wallpaper.jpg 897946


In [15]:
print(ceil(tor2.files[0].size / 524288))
print(tor2.files[0].size / 524288)
print(tor2.files[0].size % 524288)

print(ceil(tor2.files[1].size / 524288))
print(tor2.files[1].size / 524288)
print(tor2.files[1].size % 524288)

5
4.169071197509766
88642
2
1.7126960754394531
373658


In [16]:
# Extract the name of the file
file_name = tor.name
print("File Name:", file_name)

# Extract the size of the file
file_size = tor.size
print("File Size:", file_size)

# Extract the number of files in the torrent
num_files = len(tor.files)
print("Number of Files:", num_files)

# Extract the trackers used in the torrent
trackers = tor.trackers
print("Trackers:", trackers)

File Name: pic.jpg
File Size: 2185794
Number of Files: 1
Trackers: []


In [17]:
torrent = Torrent.read("pic.torrent")
files = {}
pointer = 0
for f in torrent.files:
    trailing_start = 0
    trailing_end = 0
    size = f.size
    name = f.name
    if pointer % torrent.piece_size != 0:
        trailing_start = torrent.piece_size - pointer % torrent.piece_size
        print("trailing_start: ", trailing_start)
    end = pointer + size
    if end % torrent.piece_size != 0:
        trailing_end = end % torrent.piece_size
        print("trailing_end: ", trailing_end)
    pointer += size
    files[name] = {"trail_start": trailing_start, "trail_end": trailing_end}

trailing_end:  88642


In [18]:
piece_idx = 0
for f in torrent.files:
    trailing_start = files[f.name]["trail_start"]
    trailing_end = files[f.name]["trail_end"]
    print("trailing_start", trailing_start)
    print("trailing_end", trailing_end)
    name = f.name
    size = f.size
    print(name)
    print("initial size", size)
    with open(name, "wb") as file:
        if trailing_start:
            size -= trailing_start
            print("after trailing start", size)
            piece_idx += 1
        while size >= torrent.piece_size:
            size -= torrent.piece_size
            print("middle", size)
            piece_idx += 1
        if trailing_end:
            size -= trailing_end
            print("after trailing end", size)

trailing_start 0
trailing_end 88642
pic.jpg
initial size 2185794
middle 1661506
middle 1137218
middle 612930
middle 88642
after trailing end 0


In [19]:
print(torrent.pieces)
for piece_idx in range(0, torrent.pieces):
    print(torrent.hashes[piece_idx])

5
b'\x7f\x9a\xc30X\x1b\xaa\xa3\x11;\x11U\x99\xcd~\xa5#\x05\xe01'
b'|\xe3\xf1\xc5=^0\xa7r\xc5\xf0\xeb$+z|\t\xe9\xbc\x08'
b'D%\x9d\xe6\x05\xac\x8fP)\x12\x05\x03\x16P\x83i\xd1\x1f\xd5\xda'
b'\xe7\xa4\x94\x05\xf9^P,\xa7\x06\xbd\r\x08\x989\xbd\x93\xfaK\xad'
b'o\xe9\xad\x8d\x17\xe5\xd2f\xb9tDv\xa0\n\xf6\xd3\xab\xf5\x82Z'


In [20]:
tor2.mode

'multifile'

In [21]:
tor2.metainfo["info"]

{'files': [{'length': 2185794,
   'path': ['turquoise_mountains_art_4k_hd_turquoise.jpg']},
  {'length': 897946, 'path': ['wallpaper.jpg']}],
 'name': 'wallpapers',
 'piece length': 524288,
 'pieces': b'\x7f\x9a\xc30X\x1b\xaa\xa3\x11;\x11U\x99\xcd~\xa5#\x05\xe01|\xe3\xf1\xc5=^0\xa7r\xc5\xf0\xeb$+z|\t\xe9\xbc\x08D%\x9d\xe6\x05\xac\x8fP)\x12\x05\x03\x16P\x83i\xd1\x1f\xd5\xda\xe7\xa4\x94\x05\xf9^P,\xa7\x06\xbd\r\x08\x989\xbd\x93\xfaK\xad\x87j\xb3\xfc\xfd\x7f\xb0\xd9\x8eA[N\xca5\xb5x\x95V\xa9c\xeaZ\xb0w\xfc\x86a9\x0e\xf25~\xc4\xa6v{\x84<g\x87'}

In [2]:
import hashlib


def _read_piece(torrent, piece_idx):
    if torrent.mode == "singlefile":
        offset = piece_idx * torrent.piece_size
        try:
            with open(torrent.name, "rb") as file:
                file.seek(offset)
                data = file.read(torrent.piece_size)
                if not data:
                    return None
                return data
        except FileNotFoundError as e:
            print(f"File not found: {e}")
            return None
        except IOError as e:
            print(f"Error reading file: {e}")
            return None

    else:
        # Handle multi-file mode (if needed)
        pass


torrent = Torrent.read("compnet.torrent")

print(torrent.name)

data = _read_piece(torrent, 2)
print(len(data))
hash = hashlib.sha1(data).digest()
if hash == torrent.hashes[2]:
    print("Piece hash matches")

524288
Piece hash matches


In [8]:
import os

torrent = Torrent.read("asm1.torrent")

info_hash = torrent.infohash



def _check_local_repo(info_hash):


    torrent_files = [f for f in os.listdir("./") if f.endswith(".torrent")]


    print(torrent_files)


    for torrent_file in torrent_files:


        torrent = Torrent.read(torrent_file)


        if torrent.infohash == info_hash:


            return torrent_file


    return None



_check_local_repo(info_hash=info_hash)

['asm1.torrent', 'compnet.torrent']


'asm1.torrent'